
# Dataset analysis with Amazon SageMaker Processing Jobs using Apache Spark


Typically a machine learning (ML) process consists of few steps. First, gathering data with various ETL jobs, then pre-processing the data, featurizing the dataset by incorporating standard techniques or prior knowledge, and finally training an ML model using an algorithm.

Often, distributed data processing frameworks such as Spark are used to pre-process data sets in order to prepare them for training. In this notebook we'll use Amazon SageMaker Processing, and leverage the power of Spark in a managed SageMaker environment to run our preprocessing workload. Then, we'll take our preprocessed dataset and train a regression model using XGBoost.

![](images/processing.jpg)


In [ ]:
import sagemaker
from time import gmtime, strftime

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()

timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

# TODO:  Clean these up
#input_raw_prefix = 'sagemaker/spark-preprocess-reviews-demo/input/raw/reviews'
output_prefix = 'amazon-reviews-spark-analyzer-{}'.format(timestamp_prefix)
#model_prefix = 'sagemaker/spark-preprocess-reviews-demo/model'
processing_job_name = 'amazon-reviews-spark-analyzer-{}'.format(timestamp_prefix)

print('Processing job name:  {}'.format(processing_job_name))

## Using Amazon SageMaker Processing to execute a SparkML job

### Downloading dataset and uploading to Amazon Simple Storage Service (Amazon S3)

Show the dataset

In [ ]:
#!aws s3 ls s3://$bucket/$input_raw_prefix/
!aws s3 ls s3://amazon-reviews-pds/parquet/

### Build a Spark container for running the preprocessing job

An example Spark container is included in the `./container` directory of this example. The container handles the bootstrapping of all Spark configuration, and serves as a wrapper around the `spark-submit` CLI. At a high level the container provides:
* A set of default Spark/YARN/Hadoop configurations
* A bootstrapping script for configuring and starting up Spark master/worker nodes
* A wrapper around the `spark-submit` CLI to submit a Spark application


After the container build and push process is complete, use the Amazon SageMaker Python SDK to submit a managed, distributed Spark application that performs our dataset preprocessing.

Build the example Spark container.

In [ ]:
!cat container/Dockerfile

In [ ]:
docker_repo = 'amazon-reviews-spark-analyzer'
docker_tag = 'latest'

In [ ]:
!docker build -t $docker_repo:$docker_tag -f container/Dockerfile ./container

Create an Amazon Elastic Container Registry (Amazon ECR) repository for the Spark container and push the image.

In [ ]:
import boto3
account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name

image_uri = '{}.dkr.ecr.{}.amazonaws.com/{}:{}'.format(account_id, region, docker_repo, docker_tag)
print(image_uri)

Create ECR repository and push docker image

In [ ]:
!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)

In [ ]:
!aws ecr describe-repositories --repository-names $docker_repo || aws ecr create-repository --repository-name $docker_repo

In [ ]:
!docker tag $docker_repo:$docker_tag $image_uri

In [ ]:
!docker push $image_uri

### Run the preprocessing job using Amazon SageMaker Processing

Next, use the Amazon SageMaker Python SDK to submit a processing job. Use the Spark container that was just built, and a SparkML script for preprocessing in the job configuration.

Review the Spark preprocessing script.

In [ ]:
cat preprocess-deequ.py

Run this script as a processing job.  You specify the command (`/opt/program/submit` for this Spark processor.)  You also need to specify one `ProcessingInput` with the `source` argument of the Amazon S3 bucket and `destination` is where the script reads this data from `/opt/ml/processing/input` (inside the Docker container.)  All local paths inside the processing container must begin with `/opt/ml/processing/`.

Also give the `run()` method a `ProcessingOutput`, where the `source` is the path the script writes output data to.  For outputs, the `destination` defaults to an S3 bucket that the Amazon SageMaker Python SDK creates for you, following the format `s3://sagemaker-<region>-<account_id>/<processing_job_name>/output/<output_name>/`.  You also give the `ProcessingOutput` value for `output_name`, to make it easier to retrieve these output artifacts after the job is run.

The arguments parameter in the `run()` method are command-line arguments in our `preprocess.py` script.

In [ ]:
from sagemaker.processing import ScriptProcessor

processor = ScriptProcessor(base_job_name='spark-amazon-reviews-analyzer',
                            image_uri=image_uri,
                            command=['/opt/program/submit'],
                            role=role,
                            instance_count=10, # instance_count needs to be > 1 or you will see the following error:  "INFO yarn.Client: Application report for application_ (state: ACCEPTED)"
                            instance_type='ml.r5.8xlarge',
                            max_runtime_in_seconds=600,
                            env={
                                'mode': 'jar',
                                'main_class': 'Main'
                            })

In [ ]:
# Inputs
s3_input_data = 's3://amazon-reviews-pds/parquet/'
print(s3_input_data)

In [ ]:
!aws s3 ls $s3_input_data

In [ ]:
# Outputs
s3_output_analyze_data = 's3://{}/{}/analyze_data'.format(bucket, output_prefix)

print(s3_output_analyze_data)

In [ ]:
# Note:  We can specify the local `preprocess.py` because we're using the SageMaker SDK.
#
#    Notes on Invoking from Lambda:
#      * However, if we use the boto3 SDK (ie. with a Lambda), we need to copy the `preprocess.py` file to S3 and specify the everything include --py-files, etc.
#      * We would need to do the following before invoking the Lambda:
#          !aws s3 cp preprocess.py s3://<location>/sagemaker/spark-preprocess-reviews-demo/code/preprocess.py
#          !aws s3 cp preprocess.py s3://<location>/sagemaker/spark-preprocess-reviews-demo/py_files/preprocess.py
#      * Then reference the s3://<location> above in the --py-files, etc.
#      * See Lambda example code in this same project for more details.
#
# Note:  See https://github.com/awslabs/amazon-sagemaker-examples/issues/994 for issues related to using /opt/ml/processing/input/ and output/
processor.run(code='preprocess-deequ.py',
                    arguments=['s3_input_data', s3_input_data,
                               's3_output_analyze_data', s3_output_analyze_data,
                              ],
                    logs=True,
                    wait=False,
)

In [ ]:
#############
# TODO:  CHANGE THE processing_job_name BELOW
#############

region = sagemaker_session.boto_region_name
processing_job_name = 'spark-amazon-reviews-analyzer-2020-03-03-02-32-21-595'


#############
# TODO:  CHANGE THE processing_job_name ABOVE ^^
#############

from IPython.core.display import display, HTML
display(HTML('<b>Review <a href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(region, processing_job_name)))

In [ ]:
#!aws sagemaker list-processing-jobs

In [ ]:
#running_processor = sagemaker.processing.ProcessingJob.from_processing_name(processing_job_name=processing_job_name,
#                                                                            sagemaker_session=sagemaker_session)
#running_processor.describe()

#### Inspect the processed dataset
Take a look at a few rows of the transformed dataset to make sure the preprocessing was successful.

In [ ]:
from IPython.core.display import display, HTML
display(HTML('<b>Review <a href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 Output Data</a> After The Spark Job Has Completed</b>'.format(bucket, output_prefix, region)))

In [ ]:
!aws s3 ls --recursive $s3_output_analyze_data/

# TODO:  Load the output csv into a DataFrame and analyze
* success-metrics/
* constraint-checks/
...